Import and load dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Use UCI covertype database
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# fetch dataset
df = fetch_ucirepo(id=31)

Preprocess the data

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Note: This UCI database explicitly states there are no missing values

X = df.data.features
y = df.data.targets
nb_classes = int(y['Cover_Type'].nunique())

# Normalize features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# One-hot encode y
y = to_categorical(y['Cover_Type'] - 1, num_classes=nb_classes)

print(X)
print(y)

        Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0       -1.297805 -0.935157 -1.482820                         -0.053767   
1       -1.319235 -0.890480 -1.616363                         -0.270188   
2       -0.554907 -0.148836 -0.681563                         -0.006719   
3       -0.622768 -0.005869  0.520322                         -0.129044   
4       -1.301377 -0.988770 -1.616363                         -0.547771   
...           ...       ...       ...                               ...   
581007  -2.012130 -0.023740  0.787408                         -0.867697   
581008  -2.029988 -0.032675  0.653865                         -0.952383   
581009  -2.047847  0.029873  0.386780                         -0.985317   
581010  -2.054990  0.128163  0.119694                         -0.985317   
581011  -2.058562  0.083486 -0.147392                         -0.985317   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       

Split into training and validation

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(464809, 54)
(116203, 54)
(464809, 7)
(116203, 7)


Create the neural network with an initial layer, a residual block, an extra skip connection, further Dense layers, and an output layer

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add, ReLU
from tensorflow.keras.models import Model

inputs = Input(shape=(54,))

# Initial Dense Layer
dense_layer1 = Dense(128, activation="relu")(inputs)

# Residual Block
dense_layer2 = Dense(128, activation="relu")(dense_layer1)
dense_layer3 = Dense(128, activation="relu")(dense_layer2)
residual_connection = Add()([dense_layer3, dense_layer1])
# dense_layer4 = Dense(64, activation='relu')(residual_connection)
dense_layer4 = ReLU()(residual_connection)

# Additional Skip Connection: Bypass the residual block
skip_connection = Dense(128)(inputs)  # Linear transformation
skip_addition = Add()([dense_layer4, skip_connection])
dense_layer5 = ReLU()(skip_addition)

# Further Dense layers
dense_layer6 = Dense(128, activation="relu")(dense_layer5)
dense_layer7 = Dense(128, activation="relu")(dense_layer6)

# Output layer for multi-class classification (7 classes)
outputs = Dense(nb_classes, activation="softmax")(dense_layer7)
model = Model(inputs=inputs, outputs=outputs)

Save your complete model to visualize with the netron app

In [5]:
model.save('prathit_model_revised.h5')

Overfitting Experiment: Train model exclusively on batch of 128 training samples until loss approaches 0

In [6]:
batch_size = 128
X_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]

In [9]:
from tensorflow.keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])
model.fit(X_train_batch, y_train_batch, epochs=200, batch_size=batch_size, verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4844 - loss: 1.4789
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5078 - loss: 1.4594
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5000 - loss: 1.4243
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5078 - loss: 1.3780
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5547 - loss: 1.3260
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5391 - loss: 1.2730
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5469 - loss: 1.2223
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5469 - loss: 1.1754
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5703 - loss: 1.1313
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5938 - loss: 1.0889
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6250 - loss: 1.0473
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6406 - los

Validation Check and Conclusions

In [10]:
train_loss, train_accuracy = model.evaluate(X_train_batch, y_train_batch, verbose=1)
val_loss, val_accuracy = model.evaluate(X_valid, y_valid, verbose=1)

print(f"Validation accuracy: {val_accuracy}")
print(f"Number of parameters: {model.count_params()}")
print(f"Final training loss: {train_loss}")
print(f"Final validation loss: {val_loss}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0275  
3632/3632 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5811 - loss: 2.5484
Validation accuracy: 0.5784015655517578
Number of parameters: 43527
Final training loss: 0.030900292098522186
Final validation loss: 2.5646493434906006
